In [1]:
import os
import sys
current_directory = os.getcwd().replace('notebooks', '')
sys.path.append(current_directory)

In [2]:
import csv
import time
from time import *
from datetime import datetime

from numpy import mean, std

from main.greedy_algorithm import greedy_solver
from main.ilp_model import ilp_solver
from main.genetic_algorithm import GeneticAlgorithm

from utils.graph_reader import read_graph_from_file, read_graph_from_rakaj
from utils.graph_plotter import plot_graph

from tqdm import trange
from alive_progress import alive_bar

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_instances(instance_lvl):
        dim_path = os.path.join("../instances/", instance_lvl)
        instance_dim = instance_lvl[-1]

        instances = {}
        for instance in sorted(os.listdir(os.path.abspath(dim_path))):

            instance_path = os.path.join(dim_path, instance)

            if dim_path.endswith("0"):
                instance_info = instance.split(".txt")[0]
            else:
                instance_info = instance.split(".clq")[0]

            instances[instance_info] = instance_path

        return instances

In [25]:
def solve(instances, greedy, ilp, genetic, ga_selection_method, ga_crossover_rate, ga_mutation_rate, ga_logger):
    """Solve the dominating set problem for a given graph.

    Args:
        instances: List of instances types to include.
        greedy: If True, the greedy algorithm will be used.
        ilp: If True, the ILP algorithm will be used.
        genetic: If True, the genetic algorithm will be used.
        ga_selection_method: The selection method to be used in the genetic algorithm.
        ga_crossover_rate: The crossover rate to be used in the genetic algorithm.
        ga_mutation_rate: The mutation rate to be used in the genetic algorithm.
        ga_logger: The logger to be used in the genetic algorithm.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    subdir = ""
    instance_dim = ""

    if instances == "level0":
        subdir = instances
        instance_dim = "0"
    else:
        subdir = instances
        instance_dim ="00"

    dim_path = os.path.join("../instances/", subdir)

    with open("../results/results_lvl" + instance_dim + str(datetime.now().strftime("_%d-%m-%Y_%H-%M")) + ".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(["Instances", "Greedy algorithm", "ILP", "Genetic algorithm"])
        instance_info = ""

        with  alive_bar(len(os.listdir(os.path.abspath((dim_path)))), force_tty="True") as bar:
            for instance in sorted(os.listdir(os.path.abspath(dim_path))):
                instance_path = os.path.join(dim_path, instance)

                if dim_path.endswith("0"):
                    instance_info = instance.split(".txt")[0]

                vertices_w, edges = read_graph_from_file(instance_path)

                num_vertices = len(vertices_w)
                num_edges = len(edges)


                if greedy:
                    start = time()
                    dominating_set, weight = greedy_solver(vertices_w, edges)
                    end = time()
                    elapsed = end - start
                    greedy_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                    # print("Greedy ", instance, " done")
                else:
                    greedy_results = 0


                if ilp:
                    start = time()
                    dominating_set, weight = ilp_solver(vertices_w, edges)
                    end = time()
                    elapsed = end - start
                    ilp_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                    #print("ILP ", instance, " done")
                else:
                    ilp_results = 0

                if genetic:
                    solutions = []
                    times = []


                    for i in range(10):
                        ga = GeneticAlgorithm(vertices_w, edges, population_size=100, chromosome_length=num_vertices, selection_method=ga_selection_method, crossover_rate=ga_crossover_rate, mutation_rate=ga_mutation_rate, n_generations=100, max_time=360, logger=ga_logger)
                        start = time()
                        solution = ga.run()
                        end = time()

                        solutions.append(solution)
                        times.append(end - start)


                        dominating_set = [s.fitness[0] for s in solutions]
                        min_index = dominating_set.index(min(dominating_set))
                        min_wds = len([i for i in solutions[min_index].chromosome if i == 1])
                        ga_results = "|Ds| = " + str(min_wds) + "\nmean = " + str(mean(dominating_set)) + "\nmax = " + str(max(dominating_set)) + "\nmin = " + str(min(dominating_set)) + "\nsd = " + str(round(std(dominating_set), 3)) +"\nmean(T) = " + str(round(mean(times), 3))

                        #print("GA ", instance, " done")
                else:
                    ga_results = 0

                bar()

                instance_info += str("\n|V| = ") + str(num_vertices) + "\n|E| = " + str(num_edges)
                writer.writerow([instance_info, greedy_results, ilp_results, ga_results])

In [26]:
test_instance = "test"
instance_lvl0 = "level0"

In [29]:
solve(instances=test_instance ,greedy=True, ilp=True, genetic=True, ga_selection_method="tournament", ga_crossover_rate=0.8, ga_mutation_rate=0.1, ga_logger=False)

|████████████████████████████████████████| 1/1 [100%] in 24.5s (0.04/s)         


In [7]:
def solve_rakaj(instances, greedy, ilp, genetic, ga_selection_method, ga_crossover_rate, ga_mutation_rate, ga_logger):
    """Solve the dominating set problem for a given graph.

    Args:
        instances: List of instances types to include.
        greedy: If True, the greedy algorithm will be used.
        ilp: If True, the ILP algorithm will be used.
        genetic: If True, the genetic algorithm will be used.
        ga_selection_method: The selection method to be used in the genetic algorithm.
        ga_crossover_rate: The crossover rate to be used in the genetic algorithm.
        ga_mutation_rate: The mutation rate to be used in the genetic algorithm.
        ga_logger: The logger to be used in the genetic algorithm.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    subdir = ""
    instance_dim = ""

    if instances == "rakaj1":
        subdir = instances
        instance_dim = "1"
    elif instances == "rakaj2":
        subdir = instances
        instance_dim = "2"

    dim_path = os.path.join("../instances/", subdir)

    with open("../results/results_lvl" + instance_dim + str(datetime.now().strftime("_%d-%m-%Y_%H-%M")) + ".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(["Instances", "Greedy algorithm", "ILP", "Genetic algorithm"])
        instance_info = ""


        with  alive_bar(len(os.listdir(os.path.abspath((dim_path)))), force_tty="True") as bar:
            for instance in sorted(os.listdir(os.path.abspath(dim_path))):
                instance_path = os.path.join(dim_path, instance)

                if dim_path.endswith("0"):
                    instance_info = instance.split(".txt")[0]
                else:
                    instance_info = instance.split(".dat_")[1]

                if instances == "level0":
                    vertices_w, edges = read_graph_from_file(instance_path)
                else:
                    vertices_w, edges = read_graph_from_rakaj(instance_path)

                num_vertices = len(vertices_w)
                num_edges = len(edges)


                if greedy:
                    start = time()
                    dominating_set, weight = greedy_solver(vertices_w, edges)
                    end = time()
                    elapsed = end - start
                    greedy_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                    #print("Greedy ", instance, " done")
                else:
                    greedy_results = 0


                if ilp:
                    start = time()
                    dominating_set, weight = ilp_solver(vertices_w, edges)
                    end = time()
                    elapsed = end - start
                    ilp_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                    #print("ILP ", instance, " done")
                else:
                    ilp_results = 0

                if genetic:
                    solutions = []
                    times = []


                    for i in range(5):
                        ga = GeneticAlgorithm(vertices_w, edges, population_size=40, chromosome_length=num_vertices, selection_method=ga_selection_method, crossover_rate=ga_crossover_rate, mutation_rate=ga_mutation_rate, n_generations=100, max_time=360, logger=ga_logger)
                        start = time()
                        solution = ga.run()
                        end = time()

                        solutions.append(solution)
                        times.append(end - start)


                        dominating_set = [s.fitness[0] for s in solutions]
                        min_index = dominating_set.index(min(dominating_set))
                        min_wds = len([i for i in solutions[min_index].chromosome if i == 1])
                        ga_results = "|Ds| = " + str(min_wds) + "\nmean = " + str(mean(dominating_set)) + "\nmax = " + str(max(dominating_set)) + "\nmin = " + str(min(dominating_set)) + "\nsd = " + str(round(std(dominating_set), 3)) +"\nmean(T) = " + str(round(mean(times), 3))

                        #print("GA ", instance, " done")
                else:
                    ga_results = 0

                bar()

                instance_info += str("\n|V| = ") + str(num_vertices) + "\n|E| = " + str(num_edges)
                writer.writerow([instance_info, greedy_results, ilp_results, ga_results])


Problem.dat_150_250_0
Problem.dat_150_250_1
Problem.dat_150_250_2
Problem.dat_150_750_0
Problem.dat_150_750_1
Problem.dat_150_750_2
Problem.dat_300_1000_0
Problem.dat_300_1000_1
Problem.dat_300_1000_2
Problem.dat_300_500_0
Problem.dat_300_500_1
Problem.dat_300_500_2
Problem.dat_50_100_0
Problem.dat_50_100_1
Problem.dat_50_100_2
Problem.dat_50_500_0
Problem.dat_50_500_1
Problem.dat_50_500_2


In [11]:
rakaj1 = "rakaj1"
rakaj2 = "rakaj2"

In [ ]:
solve_rakaj(instances=rakaj1 ,greedy=True, ilp=True, genetic=True, ga_selection_method="tournament", ga_crossover_rate=0.8, ga_mutation_rate=0.1, ga_logger=False)